# Recursive Neural Networks for Sentimental Analysis

## 1. Reding the data from the csv file

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('shuffled_movie_data.csv')
df.tail()

,review,sentiment
49995,"OK, lets start with the best. the building. al...",0
49996,The British 'heritage film' industry is out of...,0
49997,I don't even know where to begin on this one. ...,0
49998,Richard Tyler is a little boy who is scared of...,0
49999,I waited long to watch this movie. Also becaus...,1


## 2. Loading Libraries

In [2]:
import numpy as np
import re
import pandas as pd
import numpy as np
from  keras.models import Sequential
from keras.layers import Activation, Dense, Embedding, SimpleRNN,Bidirectional
from keras import backend as K
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from keras.callbacks import TensorBoard

/home/alvaro/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## 3. Pre-processing data

In [3]:
from keras.preprocessing.text import Tokenizer

In [4]:
num_words = 10000
tokenizer  = Tokenizer(num_words = num_words)
tokenizer.fit_on_texts( df.review )
sequences = tokenizer.texts_to_sequences(df.review)
y  =  np.array((df.sentiment))
y[0:5]


array([1, 0, 0, 1, 0])

In [5]:
from keras.preprocessing.sequence import pad_sequences 

max_review_length = 200

pad = 'pre'

X = pad_sequences(sequences,max_review_length,padding=pad,truncating=pad)

In [6]:
#X[0:1]

## 4. Splitting data

In [7]:
from sklearn.model_selection import train_test_split

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)
print(X_train.shape)
print(X_test.shape)
input_shape = X_train.shape

(40000, 200)
(10000, 200)


## 5. Generating model

In [9]:
from keras.layers import LSTM

K.clear_session()

lstm_model = Sequential()
# We specify the maximum input length to our Embedding layer
# so we can later flatten the embedded inputs
lstm_model.add(Embedding(num_words, 
                        32, 
                        input_length=max_review_length))

lstm_model.add(Bidirectional(LSTM(32)))
lstm_model.add(Dense(1))
lstm_model.add(Activation('sigmoid'))
lstm_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 200, 32)           320000    
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                16640     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 65        
_________________________________________________________________
activation_1 (Activation)    (None, 1)                 0         
Total params: 336,705
Trainable params: 336,705
Non-trainable params: 0
_________________________________________________________________


## 6. Training Network

In [10]:
lstm_model.compile(optimizer="adam", 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

lstm_history = lstm_model.fit(X_train, 
                              y_train,
                              epochs=10,
                              batch_size=128,
                              validation_split=0.3)

Train on 28000 samples, validate on 12000 samples
Epoch 1/10
28000/28000 [==============================] - 35s 1ms/step - loss: 0.4587 - acc: 0.7751 - val_loss: 0.3113 - val_acc: 0.8709
Epoch 2/10
28000/28000 [==============================] - 34s 1ms/step - loss: 0.2487 - acc: 0.9036 - val_loss: 0.2805 - val_acc: 0.8843
Epoch 3/10
28000/28000 [==============================] - 35s 1ms/step - loss: 0.1905 - acc: 0.9317 - val_loss: 0.2866 - val_acc: 0.8813
Epoch 4/10
28000/28000 [==============================] - 37s 1ms/step - loss: 0.1474 - acc: 0.9498 - val_loss: 0.3286 - val_acc: 0.8776
Epoch 5/10
28000/28000 [==============================] - 39s 1ms/step - loss: 0.1212 - acc: 0.9597 - val_loss: 0.3370 - val_acc: 0.8754
Epoch 6/10
28000/28000 [==============================] - 39s 1ms/step - loss: 0.1102 - acc: 0.9623 - val_loss: 0.3835 - val_acc: 0.8721
Epoch 7/10
28000/28000 [==============================] - 42s 2ms/step - loss: 0.0818 - acc: 0.9753 - val_loss: 0.4245 - val_acc

## 7. Prediction fase

In [11]:
y_pred=[]
print(len(X_test)," Iterations will be done.")
for i in range(len(X_test)):
    result2 = lstm_model.predict(X_test[i].reshape(1,X_test.shape[1]),batch_size=1,verbose = 2)[0]
    pred=(result2>0.5)*1
    y_pred.append(pred)
    if i%100==0:
        print("Testeo:",i,"-> y_pred:",pred,"-> REAL:",y_test[i])

10000  Iterations will be done.
Testeo: 0 -> y_pred: [0] -> REAL: 0
Testeo: 100 -> y_pred: [1] -> REAL: 1
Testeo: 200 -> y_pred: [0] -> REAL: 1
Testeo: 300 -> y_pred: [1] -> REAL: 0
Testeo: 400 -> y_pred: [1] -> REAL: 1
Testeo: 500 -> y_pred: [1] -> REAL: 1
Testeo: 600 -> y_pred: [1] -> REAL: 1
Testeo: 700 -> y_pred: [0] -> REAL: 0
Testeo: 800 -> y_pred: [0] -> REAL: 0
Testeo: 900 -> y_pred: [1] -> REAL: 0
Testeo: 1000 -> y_pred: [1] -> REAL: 1
Testeo: 1100 -> y_pred: [1] -> REAL: 1
Testeo: 1200 -> y_pred: [1] -> REAL: 1
Testeo: 1300 -> y_pred: [0] -> REAL: 0
Testeo: 1400 -> y_pred: [1] -> REAL: 1
Testeo: 1500 -> y_pred: [0] -> REAL: 0
Testeo: 1600 -> y_pred: [1] -> REAL: 1
Testeo: 1700 -> y_pred: [0] -> REAL: 0
Testeo: 1800 -> y_pred: [1] -> REAL: 1
Testeo: 1900 -> y_pred: [0] -> REAL: 0
Testeo: 2000 -> y_pred: [0] -> REAL: 0
Testeo: 2100 -> y_pred: [1] -> REAL: 1
Testeo: 2200 -> y_pred: [1] -> REAL: 1
Testeo: 2300 -> y_pred: [0] -> REAL: 1
Testeo: 2400 -> y_pred: [1] -> REAL: 0
Teste

## 8. Results

In [12]:
from sklearn.metrics import confusion_matrix
Result=confusion_matrix(y_test, y_pred)
print(Result)

[[4133  860]
 [ 529 4478]]


In [14]:
print("Negative precision: ",Result[0,0]/(Result[0,0]+Result[0,1]))
print("Positive precision: ",Result[1,1]/(Result[1,0]+Result[1,1]))
print("General precision: ",(Result[1,1]+Result[0,0])/(Result[0,0]+Result[0,1]+Result[1,0]+Result[1,1]))

Negative precision:  0.8277588624073703
Positive precision:  0.8943479129219093
General precision:  0.8611
